<h1 align='center'> CSC4120 Programming Assignment 2  </h1>

## Submission Requirements

   The submission <font color = #942F2C>deadline is October 5 (Sun.), 2025, 11:59 pm</font>. Solutions submitted after the deadline will be graded as 0 point. Please submit an **ipynb** file and clearly state your group members' student IDs. Otherwise, your points will be deducted.

## What you need to do

1. Understand van Emde Boas trees and AVL trees.

2. Implement required functions of both data structures.

# Student IDs
Group13

122090437 QIU Runheng
122090669 YE Shuhuan
122090872 JIN Yiyao

\###

In [ ]:
import math
import cProfile
import random
from cmath import inf

# 1. van Emde Boas Trees

## Background
A van Emde Boas tree, also known as a vEB tree, implements an associative array (dictionary) with n-bit integer keys among the universe $ \{0, 1, \cdots, u - 1\} $. It performs all operations (*insert, delete, successor*...) in $ O(\log \log u) $ time.

A vEB tree for a universe with size u is implemented recursively by dividing the universe into $ \left\lceil \sqrt{u} \right\rceil $ size-$ \left\lceil \sqrt{u} \right\rceil $ vEB trees, where the $ i^{th} $ subtree stores keys among $ \{i\sqrt{u}, \cdots, (i + 1)\sqrt{u} - 1\} $.

A vEB tree stores following information:
* **u** to store the universe size.
* **min** to store the minimum element. It is stored outside the structure.
* **max** to store the maximum element.
* **cluster** array of size $ \left\lceil \sqrt{u} \right\rceil $ that points to children vEB trees.
* **summary**, an auxiliary vEB tree to keep track of emptiness of subtree/cluster.




## What you need to do

Implement insert and predecessor.

In [ ]:
# implementation of the structure

class VEBTree():
    def __init__(self, u):
        self.u = u
        self.min = None
        self.max = None
        if u <= 2:
            self.summary = None
            self.clusters = None
        else:
            self.summary = VEBTree(math.ceil(math.sqrt(u)))
            self.clusters = [VEBTree(math.ceil(math.sqrt(u))) for _ in range(math.ceil(math.sqrt(u)))]

    def high(self, x):
        return math.floor(x / math.ceil(math.sqrt(self.u)))

    def low(self, x):
        return x % math.ceil(math.sqrt(self.u))

    def index(self, high, low):
        return high * math.ceil(math.sqrt(self.u)) + low

    def VEB_min(self):
        return self.min

    def VEB_extract_min(self):
        ret = self.VEB_min()
        self.VEB_delete(ret)
        return ret

    def VEB_max(self):
        return self.max

    def is_empty(self):
        return self.min is None

    def VEB_find(self, x):
        # find x in a VEB tree, return True/False
        if self.u <= x:
            return False
        if self.min == x or self.max == x:
            return True
        if self.u == 2: 
            # if the size is 2, x must be min or max
            return False
        return self.clusters[self.high(x)].VEB_find(self.low(x))

    def VEB_insert(self, x):
        '''
        insert x into a vEB tree
        if u == 2, it is redundant to create subtrees 
        since it already stores min and max
        '''
        #######################
        #                     #
        #     TO IMPLEMENT    #
        #                     #
        #######################
        # if tree is empty, x becomes both min and max
        if self.is_empty():
            self.min = self.max = x
            return
        
        # ensure min <= x
        if x < self.min:
            x, self.min = self.min, x
        
        # base case
        if self.u == 2:
            self.max = max(x, self.max) if self.max is not None else x
            return
        
        high = self.high(x)
        low = self.low(x)
        
        # if the cluster is empty, need to insert into summary
        if self.clusters[high].is_empty():
            self.summary.VEB_insert(high)
            self.clusters[high].min = self.clusters[high].max = low
        else:
            self.clusters[high].VEB_insert(low)
        
        # update max if necessary
        if x > self.max:
            self.max = x

    def VEB_delete(self, x):
        # if not self.contains(x):
        #    print("ERROR: %d is not in the tree!"%x)
        #    return
        
        # assume x is in the tree
        
        # if the tree has only one element, directly delete it
        if self.min == self.max:
            self.min = self.max = None
        # base case where the size is 2. It must have 2 distinct elements (0, 1).
        # delete one of them
        elif self.u == 2:
            if x == 0:
                self.min = self.max = 1
            else:
                self.min = self.max = 0
        # general case
        else:
            # find new min
            if x == self.min:
                i = self.summary.min
                # i must exist since the tree must have more than 2 elements to enter this block
                x = self.min = self.index(i, self.clusters[i].min)
            # delete x
            high = self.high(x)
            self.clusters[high].VEB_delete(self.low(x))
            # if the cluster becomes empty, update summary
            if self.clusters[high].is_empty():
                self.summary.VEB_delete(high)
            # update max if the previous max is deleted
            if x == self.max:
                # has only one element 
                if self.summary.max is None:
                    self.max = self.min
                else:
                    i = self.summary.max
                    self.max = self.index(i, self.clusters[i].max)

    def VEB_successor(self, x):
        # return the successor of x (the smallest element larger than x)
        # base case 
        if self.is_empty():
            return None
        # if u == 2, has successor only if x = 0 and contains 1
        if self.u == 2:
            if x == 0 and self.max == 1:
                return 1
            return None
        # general case
        if x < self.min:
            return self.min
        if x >= self.max:
            return None
        i = self.high(x)
        if (not self.clusters[i].is_empty()) and self.low(x) < self.clusters[i].max:
            j = self.clusters[i].VEB_successor(self.low(x))
        else:
            i = self.summary.VEB_successor(i)
            if i is None:
                return None
            j = self.clusters[i].min
        return self.index(i, j)

    def VEB_predecessor(self, x):
        '''
        return the predecessor of x (the largest element smaller than x)
        '''
        #######################
        #                     #
        #     TO IMPLEMENT    #
        #                     #
        #######################
        # base case
        if self.is_empty():
            return None
        
        if self.u == 2:
            if x == 1 and self.min == 0:
                return 0
            return None
        
        # general case
        if x > self.max:
            return self.max
        
        if x <= self.min:
            return None
        
        i = self.high(x)
        
        # if current cluster has elements smaller than low(x)
        if (not self.clusters[i].is_empty()) and self.low(x) > self.clusters[i].min:
            j = self.clusters[i].VEB_predecessor(self.low(x))
        else:
            # need to look in previous clusters
            i = self.summary.VEB_predecessor(i)
            if i is None:
                # no previous cluster, check if min is the predecessor
                if self.min < x:
                    return self.min
                return None
            j = self.clusters[i].max
        return self.index(i, j)

# 2. AVL Trees

## Background

An AVL tree is a self-balancing binary search tree (BST) where the difference between heights of left and right subtrees of any node cannot be larger than one. This invariant ensures the height of the tree is $ O(\log n) $ where $ n $ is the number of nodes in the tree. Tree operations (*insertion, deletion*...) take $ O(\log n) $ time.

After insertion or deletion, the tree may become unbalanced. The invariant can be restored through one or more tree rotations.

## What you need to do

Implement insert and right-rotate.


In [ ]:
class Node():
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.height = 0

class AVLTree():
    def __init__(self) -> None:
        pass

    def AVL_insert(self, root, key):
        '''
        insert key into an AVL tree rooted at node root.
        1. first do a normal BST insertion
        2. update height
        3. calculate balance factor
        4. handle unbalance if necessay
            let z be the lowest node violating the invariant, 
            we have 4 cases:
            * right-right: z's right child y is higher (heavy) than left child, 
                and y is right-heavy or balanced -- left-rotate z.
            * right-left: z is right-heavy, its right child y is left-heavy
                -- right-rotate y then left-rotate z.
            * left-left: z is left-heavy, its left child y is left-heavy or balanced 
                -- right-rotate z.
            * left-right: z is left-heavy, its left child y is right-heavy
                -- left-rotate y then right-rotate z.
        '''
        #######################
        #                     #
        #     TO IMPLEMENT    #
        #                     #
        #######################
        # normal bst insertion
        if not root:
            return Node(key)
        elif key < root.key:
            root.left = self.AVL_insert(root.left, key)
        elif key > root.key:
            root.right = self.AVL_insert(root.right, key)
        else:
            return root
        
        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))
        
        # calculate balance
        balance = self.get_height(root.left) - self.get_height(root.right)
        
        # right-right
        if balance < -1 and self.get_balance(root.right) <= 0:
            return self.left_rotate(root)
        
        # right-left
        if balance < -1 and self.get_balance(root.right) > 0:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)
        
        # left-left
        if balance > 1 and self.get_balance(root.left) >= 0:
            return self.right_rotate(root)
        
        # left-right
        if balance > 1 and self.get_balance(root.left) < 0:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)
        
        return root

    def AVL_delete(self, root, key):
        # normal BST deletion
        if not root:
            return root
        elif key < root.key:
            root.left = self.AVL_delete(root.left, key)
        elif key > root.key:
            root.right = self.AVL_delete(root.right, key)
        else:
            if not root.left:
                ret = root.right
                root = None
                return ret
            if not root.right:
                ret = root.left
                root = None
                return ret
            newRoot = self.find_min_node(root.right)
            root.key = newRoot.key
            root.right = self.AVL_delete(root.right, newRoot.key)

        if not root:
            return root

        # update height and calculate balance factor
        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))     
        
        balance = self.get_height(root.left) - self.get_height(root.right)
            
        # handle unbalance
        # right-right
        if balance < -1 and self.get_balance(root.right) <= 0:
            return self.left_rotate(root)

        # right-left
        if balance < -1 and self.get_balance(root.right) > 0:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        # left-left
        if balance > 1 and self.get_balance(root.left) >= 0:
            return self.right_rotate(root)

        # left-right
        if balance > 1 and self.get_balance(root.left) < 0:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        return root

    '''
    Rotations to handle tree acrobatics
             y        right-rotate(y)       x
           /   \     --------------->     /   \ 
          x    T3    <---------------    T1    y
        /   \         left-rotate(x)         /   \ 
       T1   T2                              T2   T3  
    '''
    def left_rotate(self, x):
        '''
        left-rotate node x
        '''
        # rotate
        y = x.right
        T2 = y.left
        y.left = x
        x.right = T2

        # update height
        x.height = 1 + max(self.get_height(x.left), self.get_height(x.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))

        # return new children to the calling parent
        return y

    def right_rotate(self, y):
        '''
        right-rotate node y
        '''
        #######################
        #                     #
        #     TO IMPLEMENT    #
        #                     #
        #######################
        x = y.left
        T2 = x.right
        x.right = y
        y.left = T2
        
        # update height
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        x.height = 1 + max(self.get_height(x.left), self.get_height(x.right))
        
        # return new children to the calling parent
        return x

    def get_height(self, node):
        # NIL has height -1
        if not node:
            return -1
        return node.height

    def get_balance(self, node):
        if not node:
            return 0
        return self.get_height(node.left) - self.get_height(node.right)

    def find_min_node(self, root):
        if not root or not root.left:
            return root
        return self.find_min_node(root.left)

    def pre_order(self, root):
        if not root:
            return
        print(root.key, end=' ')
        self.pre_order(root.left)
        self.pre_order(root.right)   

    def AVL_min(self, root):
        return self.find_min_node(root)

    def AVL_extract_min(self, root):
        ret = self.AVL_min(root)
        return self.AVL_delete(root, ret.key), ret

    def AVL_find(self, root, key):
        # find the node with given key
        if not root:
            return None
        if key < root.key:
            return self.AVL_find(root.left, key)
        if key > root.key:
            return self.AVL_find(root.right, key)
        return root

    def AVL_successor(self, root, key):
        # find the successor node of given key
        # assume key exists in the tree
        traversed = []
        while root:
            if key == root.key:
                break
            if key < root.key:
                traversed.append([root, 'left'])
                root = root.left
            else:
                traversed.append([root, 'right'])
                root = root.right
        if not root:
            return Node(None)
        if root.right != None:
            return self.find_min_node(root.right)
        for i in reversed(range(len(traversed))):
            if traversed[i][1] == 'left':
                return traversed[i][0]
        return Node(None)
        

# 3. Linked List

## What you need to do

Jump to Section 7.

In [ ]:
# a simple linked list

class LLNode():
    def __init__(self, key):
        self.key = key
        self.next = None

class LinkedList():
    def __init__(self):
        self.head = None

    def LL_insert(self, key):
        if not self.head:
            self.head = LLNode(key)
        else:
            cur = self.head
            while cur.next:
                cur = cur.next
            cur.next = LLNode(key)
        # self.LL_print()

    def LL_delete(self, key):
        if not self.head:
            raise IndexError("delete from empty linked list")
        if self.head.key == key:
            self.head = self.head.next
            # self.LL_print()
            return
        cur = self.head
        while cur.next:
            if cur.next.key == key:
                break
            cur = cur.next
        if cur.next is None:          
            raise ValueError("delete non-exsiting element from linked list")
        cur.next = cur.next.next
        # self.LL_print()

    def LL_min(self):
        if self.head is None:
            raise ValueError("get min from empty linked list")
        ret = inf
        cur = self.head
        while cur:
            if cur.key < ret:
                ret = cur.key
            cur = cur.next
        return ret

    def LL_extract_min(self):
        ret = self.LL_min()
        self.LL_delete(ret)
        return ret

    def LL_find(self, key):
        cur = self.head
        while cur:
            if cur.key == key:
                return key
            cur = cur.next
        return None

    def LL_successor(self, key):
        ret = inf
        cur = self.head
        while cur:
            if cur.key > key and cur.key < ret:
                ret = cur.key
            cur = cur.next
        return ret

    def LL_print(self):
        cur = self.head
        while cur:
            print(cur.key, "-->", end=' ')
            cur = cur.next
        print("NIL")
        
        

# 4. Linear Array

In [ ]:
class LinearArray():
    def __init__(self):
        self.lst = []
        self.size = 0
        # visit https://wiki.python.org/moin/TimeComplexity 
        # to check the complexity of python list operations

    def LA_insert(self, x):
        if self.size < len(self.lst):
            self.lst[self.size] = x
        else:
            self.lst.append(x) # amortized worst case O(1)
        self.size += 1

    def LA_delete(self, x):
        idx = self._find(x)
        if idx == -1:
            raise ValueError("delete non-existing element")
        for i in range(idx, self.size - 1):
            self.lst[i] = self.lst[i + 1]
        self.size -= 1
        # may waste space.
        # try to avoid frequently re-allocate memory,
        # want to make insert O(1) in our test environment

    def LA_min(self):
        ret = inf
        for i in range(0, self.size):
            if self.lst[i] < ret:
                ret = self.lst[i]
        return ret

    def LA_extract_min(self):
        if self.size == 0:
            raise ValueError("extract min from empty array") 
        '''ret = inf
        idx = 0
        for i in range(0, self.size):
            if self.lst[i] < ret:
                ret = self.lst[i]
                idx = i
        for i in range(idx, self.size - 1):
            self.lst[i] = self.lst[i + 1]
        self.size -= 1'''
        ret = self.LA_min()
        self.LA_delete(ret)
        return ret

    def LA_find(self, x):
        # return the index of x in the list
        # return -1 if x is not in the list
        for i in range(self.size):
            if self.lst[i] == x:
                return i
        return -1

    def _find(self, x):
        # avoid calling LA_find in deletion
        # want to make sure functions of different data structures
        # are called same number of times
        for i in range(self.size):
            if self.lst[i] == x:
                return i
        return -1
        
    def LA_successor(self, x):
        ret = inf
        for i in range(0, self.size):
            if self.lst[i] > x and self.lst[i] < ret:
                ret = self.lst[i]
        return ret

    def LA_get_value(self, idx):
        # return the element at given idx
        if idx < 0 or idx >= self.size:
            raise IndexError("array index out of range")
        return self.lst[idx]

    def LA_get_list(self):
        return self.lst[ : self.size]



# 5. Sorted Array

In [ ]:
class SortedArray():
    def __init__(self):
        self.lst = []
        self.size = 0
        # visit https://wiki.python.org/moin/TimeComplexity 
        # to check the complexity of python list operations

    def SA_insert(self, x):
        if not self.size < len(self.lst):
            self.lst.append(x) # amortized worst case O(1)
        pos = self.size
        self.size += 1
        for i in range(self.size - 1):
            if self.lst[i] > x:
                pos = i
                break
        for i in reversed(range(pos + 1, self.size)):
            self.lst[i] = self.lst[i - 1]
        self.lst[pos] = x 

    def SA_delete(self, x):
        idx = self._find(x)
        if idx == -1:
            raise ValueError("delete non-existing element")
        for i in range(idx, self.size - 1):
            self.lst[i] = self.lst[i + 1]
        self.size -= 1
        # may waste space.
        # try to avoid frequently re-allocate memory,
        # want to make insert O(1) in our test environment

    def SA_min(self):
        return self.lst[0]

    def SA_extract_min(self):
        if self.size == 0:
            raise ValueError("extract min from empty array")
        '''ret = self.lst[0]
        for i in range(self.size - 1):
            self.lst[i] = self.lst[i + 1]
        self.size -= 1'''
        ret = self.SA_min()
        self.SA_delete(ret)
        return ret

    def _find(self, x):
        low, high = 0, self.size - 1
        while low <= high:
            mid = (low + high) // 2
            if self.lst[mid] == x:
                return mid
            elif self.lst[mid] < x:
                low = mid + 1
            else:
                high = mid - 1
        return -1

    def SA_find(self, x):
        # return the index of x in the list
        # return -1 if x is not in the list
        low, high = 0, self.size - 1
        while low <= high:
            mid = (low + high) // 2
            if self.lst[mid] == x:
                return mid
            elif self.lst[mid] < x:
                low = mid + 1
            else:
                high = mid - 1
        return -1
        
    def SA_successor(self, x):
        if x > self.lst[self.size - 1]:
            return inf
        low, high = 0, self.size - 1
        while low <= high:
            mid = (low + high) // 2
            if self.lst[mid] == x:
                if mid == self.size - 1:
                    return inf
                return self.lst[mid + 1]
            elif self.lst[mid] < x:
                low = mid + 1
            else:
                high = mid - 1
        if low >= len(self.lst):
            return inf
        return self.lst[low]

    def SA_get_value(self, idx):
        # return the element at given idx
        if idx < 0 or idx >= self.size:
            raise IndexError("array index out of range")
        return self.lst[idx]

    def SA_get_list(self):
        return self.lst[ : self.size]

# 6. Heap

Python provides built-in heap queue algorithms. Click to view the [API documentation](https://docs.python.org/3/library/heapq.html) and the [source code](https://github.com/python/cpython/blob/3.10/Lib/heapq.py).


In [ ]:
import heapq

# We implement a min-heap use 0-based indexing

'''
# Python built-in heapq functions
hq = [5, 4, 3, 2, 1]
heapq.heapify(hq)
print("The created heap is", list(hq))
heapq.heappush(hq, 0)
print("The heap after pushing 0 is",list(hq))
print("Pop the smallest item", heapq.heappop(hq))
print("Pop the smallest item", heapq.heappop(hq))
'''

class MinHeap():
    def __init__(self):
        self.heap = []

    def heap_is_empty(self):
        return len(self.heap) == 0

    def heap_insert(self, item):
        self.heap.append(item)
        self.sift_up(len(self.heap) - 1)

    def heap_min(self):
        return self.heap[0]

    def heap_extract_min(self):
        '''last = self.heap.pop() # list.pop() will raise error if list is empty
        if self.heap: # if the original heap contains more than one item
            ret = self.heap_min()
            self.heap[0] = last
            self.sift_down(0)
            return ret
        return last'''
        ret = self.heap_min()
        self.heap_delete(ret)
        return ret

    def heap_delete(self, x):
        pos = self._find(x)
        # find runs in O(n) while below deletion process runs in O(log n)
        if pos == -1:
            raise ValueError("delete non-existing element")
        if pos == len(self.heap) - 1:
            self.heap.pop()
            return
        self.heap[pos], self.heap[len(self.heap) - 1] = self.heap[len(self.heap) - 1], self.heap[pos]
        self.heap.pop()
        if self.heap:
            if pos == 0:
                self.sift_down(pos)
            elif self.heap[pos] < self.heap[(pos - 1) // 2]:
                self.sift_up(pos)
            elif self.heap[pos] > self.heap[(pos - 1) // 2]:
                self.sift_down(pos)


    def heap_successor(self, x):
        ret = inf
        for i in range(0, len(self.heap)):
            if self.heap[i] > x and self.heap[i] < ret:
                ret = self.heap[i]
        return ret

    def heap_find(self, x):
        for i in range(len(self.heap)):
            if self.heap[i] == x:
                return i
        return -1

    def _find(self, x):
        for i in range(len(self.heap)):
            if self.heap[i] == x:
                return i
        return -1

    def heap_heapify(self, lst):
        # transform list into a heap, in-place, in O(len(lst)) time
        self.heap = lst
        for i in reversed(range(len(lst) // 2)):
            self.sift_down(i)

    # sift up the item at pos
    def sift_up(self, pos):
        newItem = self.heap[pos]
        while pos > 0:
            parentPos = (pos - 1) // 2
            parent = self.heap[parentPos]
            if newItem < parent:
                self.heap[pos] = parent
                pos = parentPos
                continue
            break
        self.heap[pos] = newItem
    
    # sift down the item at pos
    def sift_down(self, pos):
        size = len(self.heap)
        startPos = pos
        newItem = self.heap[pos]
        childPos = 2 * pos + 1 # left child
        while childPos < size:
            # set childPos to the smaller child
            rightChildPos = childPos + 1
            if rightChildPos < size and self.heap[rightChildPos] < self.heap[childPos]:
                childPos = rightChildPos
            if self.heap[pos] > self.heap[childPos]:
                self.heap[pos], self.heap[childPos] = self.heap[childPos], self.heap[pos]
            pos = childPos
            childPos = 2 * pos + 1


# 7. Compare running time

In this section, we compare the running time of multiple operations (insert, delete, find, min, extract_min, successor) of the data structures we discussed before (linked list, linear array, sorted array, heap, AVL tree and vEB tree). Let the universe be integers from 0 to 9999. We first build each structure with 5000 numbers randomly selected from the universe. We then randomly select one number from the universe and do the operations. This process is repeated for 20000 times and we compare the running time.

## What you need to do
Run the code, find the running time (cumtime) of the operations, and fill in the table. Explain what you find (if you are willing to do so).

| operation   | Linked List  | Linear array | Sorted array |  Heap   | AVL tree | VEB tree  |
| ------      | -------      | -------      |  ----        | ------- | -------- | --------- |
| delete      | 19.651s      | 10.974s      |  13.232s     | 4.194s  | 1.229s   | 1.028s    |         
| extract_min | 19.012s      | 10.066s      |  8.807s      | 0.345s  | 0.704s   | 0.595s    |
| find        | 6.535s       | 3.751s       |  0.073s      | 3.703s  | 0.138s   | 0.335s    |      
| insert      | 15.031s      | 0.094s       |  16.838s     | 0.222s  | 1.539s   | 1.120s    |
| min         | 16.598s      | 10.205s      |  0.024s      | 0.021s  | 0.242s   | 0.015s    |
| successor   | 10.383s      | 7.398s       |  0.091s      | 7.066s  | 0.204s   | 0.544s    |

1. Linked List: The linked list performs the worst overall. Since operations like find, delete, and min require linear traversal, they take a long time. Both insertion and deletion are slow. The time complexity is O(n).

2. Linear Array: Insertion is fast because new elements can be appended easily, but find, delete, and extract_min are slow since the array is unsorted and requires scanning. The time complexity is O(n).

3. Sorted Array: find and min are very fast due to ordering. However, insert and delete are extremely slow because maintaining sorted order requires shifting many elements. The time complexity is O(n).

4. Heap: The heap shows good performance for insert, delete, extract_min, and min. However, successor is slow because heaps do not efficiently support ordered traversal. The time complexity is O(n).

5. AVL Tree: The AVL tree offers balanced performance for all operations. Each operation takes roughly between 0.1 and 1.5 seconds, consistent with its logarithmic complexity. The time complexity is O(log n).

6. van Emde Boas (vEB) Tree: The vEB tree achieves excellent performance. This aligns with its theoretical O(loglogU) time complexity, where U is the size of the universe. It is among the fastest data structures overall.

In [12]:
random.seed = 1

# This assignment intends to compare different data structures
# It is not difficult 
# This function may take some time to run
# So, I hope you enjoy your National Day holiday (and this course, of course)

def compare():  
    myVEBTree = VEBTree(10000)
    myAVLTree = AVLTree()
    AVLRoot = None
    myLinkedList = LinkedList()
    myLinearArray = LinearArray()
    mySortedArray = SortedArray()
    myHeap = MinHeap()

    universe = [i for i in range(10000)]
    random.shuffle(universe)

    for num in universe[ : 5000]:
        myVEBTree.VEB_insert(num)
        AVLRoot = myAVLTree.AVL_insert(AVLRoot, num)
        myLinkedList.LL_insert(num)
        myLinearArray.LA_insert(num)
        mySortedArray.SA_insert(num)
        myHeap.heap_insert(num)

    print("Data structures initialized")
 
    for _ in range(20000):
        num = random.choice(universe)

        # find
        exist = myVEBTree.VEB_find(num)
        myAVLTree.AVL_find(AVLRoot, num)
        myLinkedList.LL_find(num)
        myLinearArray.LA_find(num)
        mySortedArray.SA_find(num)
        myHeap.heap_find(num)

        # min
        VEBMin = myVEBTree.VEB_min()
        AVLMin = myAVLTree.AVL_min(AVLRoot).key
        LLMin = myLinkedList.LL_min()
        LAMin = myLinearArray.LA_min()
        SAMin = mySortedArray.SA_min()
        heapMin = myHeap.heap_min()

        # extract_min
        # all extract_mins perform in following way:
        #  find call min, then call delete(min)
        myVEBTree.VEB_extract_min()
        AVLRoot, _ = myAVLTree.AVL_extract_min(AVLRoot)
        myLinkedList.LL_extract_min()
        myLinearArray.LA_extract_min()
        mySortedArray.SA_extract_min()
        myHeap.heap_extract_min()

        # insert back
        myVEBTree.VEB_insert(VEBMin)
        AVLRoot = myAVLTree.AVL_insert(AVLRoot, AVLMin)
        myLinkedList.LL_insert(LLMin)
        myLinearArray.LA_insert(LAMin)
        mySortedArray.SA_insert(SAMin)
        myHeap.heap_insert(heapMin)

        # delete and insert
        if exist: # if the num exists in the structures
            myVEBTree.VEB_delete(num)
            AVLRoot = myAVLTree.AVL_delete(AVLRoot, num)
            myLinkedList.LL_delete(num)
            myLinearArray.LA_delete(num)
            mySortedArray.SA_delete(num)
            myHeap.heap_delete(num) 

            myVEBTree.VEB_insert(num)
            AVLRoot = myAVLTree.AVL_insert(AVLRoot, num)
            myLinkedList.LL_insert(num)
            myLinearArray.LA_insert(num)
            mySortedArray.SA_insert(num)
            myHeap.heap_insert(num)

        else:
            myVEBTree.VEB_insert(num)
            AVLRoot = myAVLTree.AVL_insert(AVLRoot, num)
            myLinkedList.LL_insert(num)
            myLinearArray.LA_insert(num)
            mySortedArray.SA_insert(num)
            myHeap.heap_insert(num) 

        # successor
        VEBSuccessor = myVEBTree.VEB_successor(num)
        AVLSuccessor = myAVLTree.AVL_successor(AVLRoot, num).key
        LLSuccessor = myLinkedList.LL_successor(num)
        LASuccessor = myLinearArray.LA_successor(num)
        SASuccessor = mySortedArray.SA_successor(num)
        heapSuccessor = myHeap.heap_successor(num)

        # delete the item if it is not in the structure previously
        if not exist:
            myVEBTree.VEB_delete(num)
            AVLRoot = myAVLTree.AVL_delete(AVLRoot, num)
            myLinkedList.LL_delete(num)
            myLinearArray.LA_delete(num)
            mySortedArray.SA_delete(num)
            myHeap.heap_delete(num)

cProfile.run("compare()", sort= 'name')


Data structures initialized
         14378991 function calls (12193124 primitive calls) in 153.876 seconds

   Ordered by: function name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  153.876  153.876 {built-in method builtins.exec}
      208    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
   379846    0.091    0.000    0.091    0.000 {built-in method builtins.len}
  1040283    0.293    0.000    0.293    0.000 {built-in method builtins.max}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
  1215644    0.168    0.000    0.168    0.000 {built-in method math.ceil}
   478415    0.074    0.000    0.074    0.000 {built-in method math.floor}
  1215644    0.348    0.000    0.348    0.000 {built-in method math.sqrt}
        2    0.000    0.000    0.000    0.000 {built-in method nt.getpid}
        1    0.000    0.000    0.000    0.000 2671455298.py:17(<listcomp>)
   6768/4    0.011    0

# 8. Bonus (10 points)

Look at the running time of your VEB functions. What sub-functions called are costly (include the python built-in functions)? Try to improve the implementation. Explain your idea and use cProfile to identify it. You can modify other lines of the class. (Hint: consider some built-in math functions).

<font color = #942F2C>Note: </font> the maximum point of this assignment is 100.

